# Etapa 2
- Calculo da Entropia e métricas.
- Criação do agente de reforço para seleção da próxima rotulação.

In [1]:

from ultralytics import YOLO
from tqdm.notebook import tqdm
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import torch

In [2]:
# Configurações
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
images_in_pool = [f"E:/COCO-Dataset/train2017/val/images/{f}" for f in os.listdir("E:/COCO-Dataset/train2017/val/images/")]
oracle_labels = {img: img.replace(".jpg",".txt").replace("images","labels") for img in images_in_pool}  # Preencher com anotações reais

In [4]:
print(images_in_pool[0])
print(oracle_labels[images_in_pool[0]])

E:/COCO-Dataset/train2017/val/images/000000000009.jpg
E:/COCO-Dataset/train2017/val/labels/000000000009.txt


In [5]:
  # 1. Carregar YOLOv11
yolo = YOLO("runs/detect/yolov11-initial/weights/best.pt").to(DEVICE)
    
from envRL import YOLORLEnv
# 3. Criar ambiente
env = YOLORLEnv(
        yolo_model=yolo,
        unlabeled_images=images_in_pool,
        oracle_labels=oracle_labels,
        batch_size=8,
)

env = DummyVecEnv([lambda: env])

# Hyperparâmetros ajustados
policy_kwargs = {
    'net_arch': {
        'pi': [256, 256],  # Rede de política
        'vf': [256, 256]   # Rede de valor
    },
    'optimizer_class': torch.optim.AdamW,
    'activation_fn': torch.nn.ReLU,
    'ortho_init': True
}

agent = PPO(
    "MlpPolicy",
    env,
    n_steps=2048,              # Tamanho do rollout
    batch_size=8,              # Tamanho do mini-batch
    gamma=0.99,                # Fator de desconto
    gae_lambda=0.95,           # Parâmetro GAE
    ent_coef=0.01,             # Coeficiente de entropia
    learning_rate=3e-4,        # Taxa de aprendizado
    clip_range=0.2,            # Clipping de política
    verbose=2,
    policy_kwargs=policy_kwargs,
    device="cuda"
)


Using cuda device


C:\Users\Gabriel\AppData\Roaming\Python\Python313\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [ ]:

agent.learn(
    total_timesteps=5000,
)
# Salvar versão final
agent.save("ppo_agent_final")

Confs: [    0.50048     0.49952]
ENTROPIA 0.6931467056274414
Confs: [    0.50048     0.49952]
ENTROPIA 0.6931467056274414
Movidos: 000000218716.jpg e label para treinamento
Confs: [          1]
ENTROPIA -0.0
Confs: [    0.53608     0.25315     0.21077]
ENTROPIA 1.0101687908172607
Confs: []
Confs: []
Movidos: 000000432843.jpg e label para treinamento
Confs: [          1]
ENTROPIA -0.0
Confs: [    0.56892     0.43108]
ENTROPIA 0.6836172342300415
Confs: [    0.25139     0.23703     0.19626     0.15321    0.088625    0.073481]
ENTROPIA 1.7019240856170654
Confs: [          1]
ENTROPIA -0.0
Confs: [    0.25679     0.22627     0.22002     0.13183    0.089244    0.075856]
ENTROPIA 1.6968560218811035
Confs: [    0.25679     0.22627     0.22002     0.13183    0.089244    0.075856]
ENTROPIA 1.6968560218811035
Movidos: 000000362547.jpg e label para treinamento
Confs: [    0.12155     0.10794    0.098344    0.082388    0.079591     0.07284    0.069629    0.059602    0.055142    0.053471    0.048296

val: Scanning E:\COCO-Dataset\val2017\labels.cache... 5000 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▍       | 77/313 [00:27<01:24,  2.78it/s]

# Saída de treinamento do algoritmo PPO (Proximal Policy Optimization) no Stable Baselines3

**n_updates** 

Número de atualizações dos parâmetros da política (rede neural) durante o treinamento.
Cada "update" corresponde a uma época de treinamento em um mini-batch de dados coletados.

**policy_gradient_loss**

Perda associada à política (a função que decide ações).
Mede quão bem a política está maximizando a vantagem esperada (recompensas futuras ajustadas pela linha de base).
- Valores negativos indicam que a política está melhorando (a vantagem está aumentando).
- Valores positivos podem indicar instabilidade ou dificuldade de convergência.

**value_loss**

Perda associada à função de valor (estimador do retorno esperado).
Mede o erro na previsão do valor esperado (recompensa acumulada futura) para um estado.
- Quanto menor, melhor a função de valor está estimando as recompensas.